In [1]:
import numpy as np
import pandas as pd
import re
import jieba

In [27]:
data=pd.read_excel("./data/data_demo_pos.xls")
data.head()

,id,pno,context_pos,diag_code,tcms_code,tan,feng,ren,yinxu,fei
0,1,1803613,右肺癌术后三月余。 患者面色憔悴；形体消瘦，；言语清晰、流畅；气息平和；舌质稍红，苔少，脉细...,0,0,0,0,0,1,1
1,2,1802607,右肺癌术后两月余。 患者面色憔悴；形体消瘦，；言语清晰、流畅；气息平和；舌质稍红，苔少，脉细...,0,0,0,0,0,1,1
2,3,1809724,右肺癌术后8年余，咳嗽气喘加重1周。 患者气喘貌，；言语清晰、流畅；气息尚平和；舌质稍红，苔...,0,0,0,0,0,1,1
3,4,1710003,右肺癌术后8年余，反复咳嗽气喘半年余。 患者气喘貌，；言语清晰、流畅；气息尚平和；舌质稍红，...,0,0,0,0,0,1,1
4,5,1613461,右肺癌术后7年余，咳嗽气喘20小时。 患者气喘貌，；言语清晰、流畅；气息尚平和；舌质稍红，苔...,0,0,0,0,0,1,1


In [3]:
y=data.tcms_code
tcme=data.iloc[:,5:]
context=data.context_pos

In [4]:
print(context.shape,y.shape,tcme.shape)

(1400,) (1400,) (1400, 5)


In [5]:
dictfile="./data/tcm_all_dict_2.txt"
stopwords="./data/stopwords.txt"
jieba.load_userdict(dictfile)  

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Tang\AppData\Local\Temp\jieba.cache
Loading model cost 1.577 seconds.
Prefix dict has been built succesfully.


In [6]:
def load_stopword(filename):  
    stopwords = [line.strip() for line in open(filename, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [7]:
def get_corpus(context,stopwords):
    stopword_list=load_stopword(stopwords) 
    corpus=[]    
    
    for sent in context:        
        strlist=list(jieba.cut(sent))
        strlist=sorted(set(strlist),key=strlist.index)
        doc_stopword=[y for y in strlist if y not in stopword_list] #drop stopword
        corpus.append(doc_stopword)
        
    return corpus

In [8]:
cps=get_corpus(context, stopwords)

In [9]:
#TextCNN - TCM syndrome

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [11]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [12]:
y=data.tcms_code
all_texts=cps
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y),num_classes=None)

Found 1077 unique tokens.


In [13]:
from sklearn.model_selection import train_test_split
#x=datatxt
#y1=labels
x_train, x_test, y_train, y_test=train_test_split(datatxt,labels,test_size=0.2,random_state=None)

In [14]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

In [15]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 256)           275968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 512)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_1 (Dense)      

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=32, epochs=8, verbose=1, validation_split=0.2, shuffle=True) 
#score1 = model.evaluate(x_train,y_train, verbose=0)
score2 = model.evaluate(x_test,y_test, verbose=0)
print("accuracy:",score2[1])

Instructions for updating:
Use tf.cast instead.
Train on 896 samples, validate on 224 samples
Epoch 1/8
896/896 [==============================] - 6s 7ms/step - loss: 0.8931 - acc: 0.6272 - val_loss: 0.5493 - val_acc: 0.7902
Epoch 2/8
896/896 [==============================] - 4s 5ms/step - loss: 0.2762 - acc: 0.8929 - val_loss: 0.3133 - val_acc: 0.8795
Epoch 3/8
896/896 [==============================] - 5s 5ms/step - loss: 0.1417 - acc: 0.9531 - val_loss: 0.3838 - val_acc: 0.8571
Epoch 4/8
896/896 [==============================] - 5s 5ms/step - loss: 0.0910 - acc: 0.9665 - val_loss: 0.2994 - val_acc: 0.8973
Epoch 5/8
896/896 [==============================] - 5s 5ms/step - loss: 0.0472 - acc: 0.9844 - val_loss: 0.3248 - val_acc: 0.9196
Epoch 6/8
896/896 [==============================] - 5s 5ms/step - loss: 0.0349 - acc: 0.9866 - val_loss: 0.3349 - val_acc: 0.9062
Epoch 7/8
896/896 [==============================] - 5s 5ms/step - loss: 0.0225 - acc: 0.9955 - val_loss: 0.3767 - val_a

In [17]:
#TextCNN - TCM element

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [19]:
MAX_SEQUENCE_LENGTH =64
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.16 
TEST_SPLIT = 0.2

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()

In [21]:
ym=pd.DataFrame(tcme)

all_texts=cps
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
datatxt = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#labels = to_categorical(np.asarray(y),num_classes=None)
print('Shape of data tensor:', datatxt.shape)
print('Shape of label tensor:', ym.shape)
print(word_index)

Found 1077 unique tokens.
Shape of data tensor: (1400, 64)
Shape of label tensor: (1400, 5)
{' ': 1, '咳嗽': 2, '咳痰': 3, '患者': 4, '为主': 5, '舌质': 6, '精神': 7, '面色': 8, '诉': 9, '结合': 10, '红': 11, '苔': 12, '舌脉': 13, '痰': 14, '清晰': 15, '阵发性': 16, '天': 17, '发热': 18, '二便': 19, '言语': 20, '伴': 21, '脉滑': 22, '胸闷': 23, '稍': 24, '气喘': 25, '正常': 26, '神清': 27, '咯痰': 28, '形体': 29, '双目': 30, '呼吸': 31, '发育': 32, '营养': 33, '应答': 34, '自如': 35, '语声': 36, '调': 37, '少神': 38, '有神': 39, '清': 40, '黄腻': 41, '食纳': 42, '少华': 43, '舌红': 44, '平稳': 45, '适中': 46, '行动自如': 47, '淡红': 48, '白腻': 49, '加重': 50, '后': 51, '表情': 52, '声音洪亮': 53, '脉浮': 54, '月余': 55, '尚可': 56, '神志': 57, '不易': 58, '活动': 59, '苔薄': 60, '咯出': 61, '气促': 62, '一周': 63, '精神不振': 64, '数': 65, '黄': 66, '脉滑数': 67, '中等': 68, '脉细': 69, '症': 70, '脉细数': 71, '响亮': 72, '苔少': 73, '肺癌': 74, '乏力': 75, '患儿': 76, '良好': 77, '余': 78, '咯': 79, '粘': 80, '欠佳': 81, '自然': 82, '三天': 83, '咯吐': 84, '粘痰': 85, '十余天': 86, '时有': 87, '伴有': 88, '月': 89, '痛苦': 90, '咳声': 91, '纳差': 92, '气息'

In [22]:
from sklearn.model_selection import train_test_split
x=datatxt
#ym=pd.DataFrame(outm)
x_train, x_test, y_train, y_test=train_test_split(x,ym,test_size=0.20,random_state=0)

In [23]:
y_test.head()

,tan,feng,ren,yinxu,fei
1110,1,0,0,0,1
310,0,0,0,1,1
776,1,0,1,0,1
491,0,1,1,0,1
1063,1,0,0,0,1


In [24]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

In [25]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(512, 3, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
#model.add(Dense(ym.shape[1], activation='softmax')) #multiclass
model.add(Dense(ym.shape[1], activation='sigmoid')) #multilabel
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 64, 256)           275968    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 512)           393728    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 512)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4194560   
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 1285      
Total para

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"]) 
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["acc"]) 
#model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_data=(x_test, y_test)) 
model.fit(x_train, y_train, batch_size=64, epochs=4, verbose=1, validation_split=0.2, shuffle=True)
score = model.evaluate(x_test,y_test, verbose=0)
print(score[1])

Train on 896 samples, validate on 224 samples
Epoch 1/4
896/896 [==============================] - 5s 6ms/step - loss: 0.4419 - acc: 0.7654 - val_loss: 0.2974 - val_acc: 0.8848
Epoch 2/4
896/896 [==============================] - 4s 4ms/step - loss: 0.2288 - acc: 0.9112 - val_loss: 0.1367 - val_acc: 0.9607
Epoch 3/4
896/896 [==============================] - 4s 4ms/step - loss: 0.1081 - acc: 0.9600 - val_loss: 0.0907 - val_acc: 0.9652
Epoch 4/4
896/896 [==============================] - 4s 4ms/step - loss: 0.0629 - acc: 0.9746 - val_loss: 0.0745 - val_acc: 0.9705
0.9742857166699
